In [2]:
# Importing the necessary modules. 
import os
import csv
import requests
import time
from bs4 import BeautifulSoup
import pandas as pd

In [38]:
url = 'https://bsnpr.com/juegos/juego?r=BS20221035'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')




In [85]:
# Define a list of team numbers and names
game_date = '20221035'

def boxscore_scrape(year,month,day):
    try:
        date_str = f"{year}{month}{day}"
        url = 'https://bsnpr.com/juegos/juego?r=BS' + str(date_str)
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')


        tables = soup.findAll("table", class_="tabla_stats")
        if len(tables)< 2:
            print(f" No boxscore found for date {date_str}")
        else:
            t_boxscore = tables[1]
            t_hometeam = tables[2]
            t_awayteam = tables[3]

        # extract the team names and scores from the table_boxscore
        team_names = []
        team_scores = []
        for row in t_boxscore.findAll("tr")[1:]:
            cols = row.findAll("td")
            team_names.append(cols[0].text.strip())
            team_scores.append(cols[-1].text.strip())

        # Extract the table data
        table_data = []
        for row in t_boxscore.findAll("tr")[1:]:
            cols = row.findAll("td")[1:-1]
            row_data = [col.text.strip() for col in cols]
            table_data.append(row_data)

        df_table = pd.DataFrame(table_data[:4], columns=['1Q', '2Q', '3Q', '4Q'])
        ## Check if the boxscore has at least 4 quarters
        if len(table_data) >= 4:
            # Create a DataFrame from the table data with 4 columns for the first 4 quarters


            # If there are more than 4 quarters, add an 'OT' column for each additional quarter
            for i in range(4, len(table_data)):
                column_name = f"OT{i-3}"  # Name the column 'OT5' for the 5th quarter, 'OT6' for the 6th, and so on
                df_table[column_name] = table_data[i]

            # Add the boxscore to the list of errors if it doesn't have at least 4 quarters
        else:
        # Create a DataFrame for the team names and scores 
            df_team = pd.DataFrame({'Team': team_names, 'Total': team_scores})

        # Concatenate the two DataFrames
            df = pd.concat([df_team, df_table], axis=1)
            df.drop(2, inplace=True)
            print(df)


        # Extract the table data fro the home team
        table_data_home = []
        for row in t_hometeam.findAll("tr")[1:]:
            cols = row.findAll("td")
            row_data = [col.text.strip() for col in cols]
            table_data_home.append(row_data)


        # Create a DataFrame for the home team stats
        df_home = pd.DataFrame(table_data_home, columns=['','Name', 'Min', 'M', 'A', 'FG%', 'M', 'A', '3P%', 'M', 'A', 'FT%', 'O', 'D', 'T', 'AS', 'PF', 'TO', 'ST', 'BS', 'PTS'])

        # add a new column team with the value of the data frame earlier
        df_home['Team'] = team_names[0]

        # Remove the empty column  
        #df_home = df_home.drop(columns=['']).iloc[1:]

        # add a prefix to the column names of df_home 


        # Extract the table data for the away team
        table_data_away = []
        for row in t_awayteam.findAll("tr")[1:]:
            cols = row.findAll("td")
            row_data = [col.text.strip() for col in cols]
            table_data_away.append(row_data)

        # Create a DataFrame for teh away team stats
        df_away = pd.DataFrame(table_data_away)
        df_away['Team'] = team_names[1]


        # Remove the empty column
        #df_away = df_away.drop(columns=['']).iloc[1:]
        folder_path = f"/Users/jeanzayas/Desktop/Divergence/Portfolio/Sports Analysis/Basketball/BSN/dataset/{year}"
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)
            game_id = date_str
            file_path = f"{folder_path}/{game_id}.csv"
            df_boxscore.to_csv(file_path, index=False)

        df_home.to_csv(f"/Users/jeanzayas/Desktop/Divergence/Portfolio/Sports Analysis/Basketball/BSN/dataset/{year}/{date_str}_home.csv", index=False)
        df_away.to_csv(f"/Users/jeanzayas/Desktop/Divergence/Portfolio/Sports Analysis/Basketball/BSN/dataset/{year}/{date_str}_away.csv", index=False)
                    
        print(df_home)
        print(df_away)

    except:
        print(game_date + 'Did not work')
        print(soup)

In [80]:
boxscore_scrape()

       Team Total  1Q  2Q  3Q  4Q
0   Arecibo    82  17  22  23  20
1  Mayaguez    67   9  13  26  19
                          Name    Min     M     A   FG%     M     A   3P%  \
0   None                  None   None  None  None  None  None  None  None   
1      2          Soto, Joseph  30:28     2     4   50%     2     3   67%   
2     10           Liz, Víctor  24:25     2     5   40%     1     5   20%   
3     12      Elonu, Chinemelu  18:48     5     7   71%     0     0    0%   
4     19        Agosto, Jeremy   1:20     0     0    0%     0     0    0%   
5     23         Gaston, Chris  25:38     8    10   80%     0     0    0%   
6     24   Rodríguez, Jonathan  22:22     0     1    0%     0     3    0%   
7     25         Boyd, Brandon   9:32     1     3   33%     2     6   33%   
8     26      Cintrón, Raymond  12:35     0     0    0%     1     4   25%   
9     33        Huertas, David  33:35     2     3   67%     3     8   38%   
10    44        Collier, Devon  16:28     1     3  

# BSN calendar pull 

1. Define a list of team numbers and names:

https://bsnpr.com/juegos/juego?cid=29382&r=BS         2022 10 35

In [86]:
for year in range(2022, 2023):
    for month in range(10, 13):
        for days in range(1, 100):
            boxscore_scrape(year, month, days)


 No boxscore found for date 2022101
20221035Did not work

<!DOCTYPE html>

<html lang="en">
<!--include file="includes/detect_device.asp" -->
<!--include file="includes/mobile_check.asp" -->
<!--include file="includes/geolocation.asp" -->
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<title>Baloncesto Superior Nacional | Calendario</title>
<!-- Bootstrap -->
<link href="../css/bootstrap-4.2.1.css" rel="stylesheet"/>
<link crossorigin="anonymous" href="https://use.fontawesome.com/releases/v5.8.1/css/all.css" integrity="sha384-50oBUHEmvpQ+1lW4y57PTFmhCaXp0ML5d60M1M7uH2+nqUivzIebhndOJK28anvf" rel="stylesheet"/>
<link href="../css/custom.css" rel="stylesheet"/>
<style></style>
<script type="text/javascript">



function getLiga() {
	var y = $("#anio").val();
	var liga = $("#liga").val();
	var serie = $("#serie").val();
	var mes = $("#mes").val();
	var dia = $("#dia").val();
	//var 

UnboundLocalError: local variable 'soup' referenced before assignment

In [ ]:
# Gather the correct libraries 
import os 
import csv 
import requests 
import time
from bs4 import BeautifulSoup
import pandas as pd


In [9]:
# Define the base URL for the boxscores
base_url = 'https://bsnpr.com/juegos/juego?r='

# Define a list of team url number, team name and team town

teams = [
    (1, 'Cangrejeros', 'Santurce'),
    (2, 'Capitanes', 'Arecibo'),
    (3, 'Atleticos', 'San German'),
    (5, 'Leones', 'Ponce'),
    (6, 'Piratas', 'Quebradillas'),
    (9, 'Vaqueros', 'Bayamon'),
    (10, 'Gigantes', 'Carolina'),
    (12, 'Indios', 'Mayaguez'),
    (14, 'Mets', 'Guaynabo'),
    (15, 'Cariduros', 'Fajardo'),
    (73, 'Grises', 'Humacao'),
    (79, 'Osos', 'Manati')
]

# Define the range of years, months, and days to scrape
years = range(2022, 2023)
months = range(10, 12)
days = range(1, 100)

In [47]:
# Loop over the years, months, and days
for year in years:
    # Create a folder for the year if it doesn't exist
    if not os.path.exists(str(year)):
        os.makedirs(str(year))
    for month in months:
        for day in days:
            # Construct the date string 
            date_str = f"{year}{month}{day}"
            #f"{year}{month:02d}{day:02d}"
            url_bs = base_url + 'BS' + date_str
            
            while True:
                # Send a request to the URL and parse the HTML
                response = requests.get(url_bs)
                if response.status_code == 200:
                    print(f'Processing date: {date_str}')
                    
                    soup = BeautifulSoup(response.content, 'html.parser')
                
                    tables = soup.findAll("table", class_="tabla_stats")
                    
                    if len(tables) < 2:
                        print(f"No boxscore found for date {date_str}")
                        break
                    else:
                        t_boxscore = tables[1]
                        t_hometeam = tables[2]
                        t_awayteam = tables[3]
                    
                    # Extract the team names and scores from the table_bs
                
                        team_towns = []
                        team_scores = []
                        for row in t_boxscore.findAll("tr")[1:]:
                            cols = row.findAll("td")
                        team_towns.append(cols[0].text.strip())
                        team_scores.append(cols[-1].text.strip())
                
                    # Extract the table data
                        final_boxscore = []
                        for row in t_boxscore.findAll("tr")[1:]:
                            cols = row.findAll("td")[1:-1]
                        row_data = [col.text.strip() for col in cols]
                        final_boxscore.append(row_data)
                    df_table = pd.DataFrame(final_boxscore[:4], columns=['1Q', '2Q', '3Q', '4Q']) 
                    # Create a DataFrame from the table data 
                    if len(final_boxscore) >=4:
                        
                         # If there are more than 4 quarters, add an 'OT' column for each additional quarter
                            
                        for i in range(4, len(final_boxscore)):
                            column_name = f"OT{i-3}"  # Name the column 'OT5' for the 5th quarter, 'OT6' for the 6th, and so
                            df_table[column_name] = final_boxscore[i]
                    else: 
                        print(final_boxscore)
                    # Create a DataFrame for the team names and scores 
                    df_team = pd.DataFrame({'Team': team_towns, 'Score': team_scores})

                    # Concatenate the two DataFrames
                    df = pd.concat([df_team, df_table], axis=1)
                    print(df.iloc[0]['Team'], "vs", df.iloc[1]['Team'])
                    
                    
                    # Save the box score file in a yearly folder with the date_str as game_id variable
                    folder_path = f"/Users/jeanzayas/Desktop/Divergence/Portfolio/Sports Analysis/Basketball/BSN/dataset/{year}"
                    if not os.path.exists(folder_path):
                        os.makedirs(folder_path)
                    game_id = date_str
                    file_path = f"{folder_path}/{game_id}.csv"
                    df.to_csv(file_path, index=False)
                    
                    team_names = []
                    team_scores = []
                    for row in t_boxscore.findAll("tr")[1:]:
                            cols = row.findAll("td")
                            team_names.append(cols[0].text.strip())
                            team_scores.append(cols[-1].text.strip())
    
                    # Extract the table data
                    table_data = []
                    for row in t_boxscore.findAll("tr")[1:]:
                                cols = row.findAll("td")[1:-1]
                                row_data = [col.text.strip() for col in cols]
                    table_data.append(row_data)

                    ## Check if the boxscore has at least 4 quarters
                    #if len(table_data) >= 4:
                        # Create a DataFrame from the table data with 4 columns for the first 4 quarters
                    df_table = pd.DataFrame(table_data[:4], columns=['1Q', '2Q', '3Q', '4Q'])
    
                        # If there are more than 4 quarters, add an 'OT' column for each additional quarter
                    for i in range(4, len(table_data)):
                                column_name = f"OT{i-3}"  # Name the column 'OT5' for the 5th quarter, 'OT6' for the 6th, and so
                                df_table[column_name] = table_data[i]
                    else:
    # Add the boxscore to the list of errors if it doesn't have at least 4 quarters
                    

    
                    # Save to CSV
                                df_home.to_csv(f"/Users/jeanzayas/Desktop/Divergence/Portfolio/Sports Analysis/Basketball/BSN/dataset/{year}/{date_str}_home.csv", index=False)
                                df_away.to_csv(f"/Users/jeanzayas/Desktop/Divergence/Portfolio/Sports Analysis/Basketball/BSN/dataset/{year}/{date_str}_away.csv", index=False)
                    
                    
                    # Exit the while loop
                    break
            
                else: 
                    print(f'No data found for BoxScore: {date_str}')


Processing date: 2022101
No boxscore found for date 2022101
Processing date: 2022102
No boxscore found for date 2022102
Processing date: 2022103
No boxscore found for date 2022103
Processing date: 2022104
No boxscore found for date 2022104
Processing date: 2022105
No boxscore found for date 2022105
Processing date: 2022106
No boxscore found for date 2022106
Processing date: 2022107
No boxscore found for date 2022107
Processing date: 2022108
No boxscore found for date 2022108
Processing date: 2022109
No boxscore found for date 2022109
Processing date: 20221010
[['', '', '', '']]


IndexError: single positional indexer is out-of-bounds

In [96]:
# Define a list of team numbers and names

teams = [
    (1, 'Cangrejeros', 'Santurce'),
    (2, 'Capitanes', 'Arecibo'),
    (3, 'Atleticos', 'San German'),
    (5, 'Leones', 'Ponce'),
    (6, 'Piratas', 'Quebradillas'),
    (9, 'Vaqueros', 'Bayamon'),
    (10, 'Gigantes', 'Carolina'),
    (12, 'Indios', 'Mayaguez'),
    (14, 'Mets', 'Guaynabo'),
    (15, 'Cariduros', 'Fajardo'),
    (73, 'Grises', 'Humacao'),
    (79, 'Osos', 'Manati')]

# extract the team names and scores from the table_boxscore
team_names = []
team_scores = []
for row in t_boxscore.findAll("tr")[1:]:
    cols = row.findAll("td")
    team_names.append(cols[0].text.strip())
    team_scores.append(cols[-1].text.strip())
    
# Extract the table data
table_data = []
for row in t_boxscore.findAll("tr")[1:]:
    cols = row.findAll("td")[1:-1]
    row_data = [col.text.strip() for col in cols]
    table_data.append(row_data)

## Check if the boxscore has at least 4 quarters
if len(table_data) >= 4:
    # Create a DataFrame from the table data with 4 columns for the first 4 quarters
    df_table = pd.DataFrame(table_data[:4], columns=['1Q', '2Q', '3Q', '4Q'])
    
    # If there are more than 4 quarters, add an 'OT' column for each additional quarter
    for i in range(4, len(table_data)):
        column_name = f"OT{i-3}"  # Name the column 'OT5' for the 5th quarter, 'OT6' for the 6th, and so
        df_table[column_name] = table_data[i]
else:
    # Add the boxscore to the list of errors if it doesn't have at least 4 quarters
    
# Create a DataFrame for the team names and scores 
    df_team = pd.DataFrame({'Team': team_names, 'Score': team_scores})

# Concatenate the two DataFrames
df = pd.concat([df_team, df_table], axis=1)

# drop last row

df.drop(2, inplace=True)

print(df.iloc[0]['Team'], df.iloc[0]['Score'], "vs", df.iloc[1]['Team'], df.iloc[0]['Score'])

# Save the box score file in a yearly folder with the date_str as game_id variable
folder_path = f"/Users/jeanzayas/Desktop/Divergence/Portfolio/Sports Analysis/Basketball/BSN/dataset/{year}"
if not os.path.exists(folder_path):
    os.makedirs(folder_path)
game_id = date_str
file_path = f"{folder_path}/{game_id}.csv"
df.to_csv(file_path, index=False)

Bayamon 84 vs Ponce 84
